In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf

if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU")
else:
    print("TensorFlow is not using GPU")

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Lambda, Activation, Concatenate, Add, Dropout
from tensorflow.keras.layers import BatchNormalization, Conv2D, DepthwiseConv2D, ZeroPadding2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.utils import get_source_inputs
from tensorflow.keras.utils import get_file
from tensorflow.keras import backend as K
from tensorflow.keras.activations import relu

# from tensorflow.python.keras.applications.imagenet_utils import preprocess_input
from sklearn.metrics import precision_score, recall_score, f1_score

WEIGHTS_PATH_X = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.1/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_MOBILE = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.1/deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels.h5"
WEIGHTS_PATH_X_CS = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.2/deeplabv3_xception_tf_dim_ordering_tf_kernels_cityscapes.h5"
WEIGHTS_PATH_MOBILE_CS = "https://github.com/bonlime/keras-deeplab-v3-plus/releases/download/1.2/deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels_cityscapes.h5"


# from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
# import os
import re
import rasterio
import glob
import tifffile as tiff


TensorFlow is using GPU


In [2]:
def SepConv_BN(x, filters, prefix, stride=1, kernel_size=3, rate=1, depth_activation=False, epsilon=1e-3):
    """ SepConv with BN between depthwise & pointwise. Optionally add activation after BN
        Implements right "same" padding for even kernel sizes
        Args:
            x: input tensor
            filters: num of filters in pointwise convolution
            prefix: prefix before name
            stride: stride at depthwise conv
            kernel_size: kernel size for depthwise convolution
            rate: atrous rate for depthwise convolution
            depth_activation: flag to use activation between depthwise & poinwise convs
            epsilon: epsilon to use in BN layer
    """

    if stride == 1:
        depth_padding = 'same'
    else:
        kernel_size_effective = kernel_size + (kernel_size - 1) * (rate - 1)
        pad_total = kernel_size_effective - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        x = ZeroPadding2D((pad_beg, pad_end))(x)
        depth_padding = 'valid'

    if not depth_activation:
        x = Activation('relu')(x)
    x = DepthwiseConv2D((kernel_size, kernel_size), strides=(stride, stride), dilation_rate=(rate, rate),
                        padding=depth_padding, use_bias=False, name=prefix + '_depthwise')(x)
    x = BatchNormalization(name=prefix + '_depthwise_BN', epsilon=epsilon)(x)
    if depth_activation:
        x = Activation('relu')(x)
    x = Conv2D(filters, (1, 1), padding='same',
               use_bias=False, name=prefix + '_pointwise')(x)
    x = BatchNormalization(name=prefix + '_pointwise_BN', epsilon=epsilon)(x)
    if depth_activation:
        x = Activation('relu')(x)

    return x


In [3]:
def _conv2d_same(x, filters, prefix, stride=1, kernel_size=3, rate=1):
    """Implements right 'same' padding for even kernel sizes
        Without this there is a 1 pixel drift when stride = 2
        Args:
            x: input tensor
            filters: num of filters in pointwise convolution
            prefix: prefix before name
            stride: stride at depthwise conv
            kernel_size: kernel size for depthwise convolution
            rate: atrous rate for depthwise convolution
    """
    if stride == 1:
        return Conv2D(filters,
                      (kernel_size, kernel_size),
                      strides=(stride, stride),
                      padding='same', use_bias=False,
                      dilation_rate=(rate, rate),
                      name=prefix)(x)
    else:
        kernel_size_effective = kernel_size + (kernel_size - 1) * (rate - 1)
        pad_total = kernel_size_effective - 1
        pad_beg = pad_total // 2
        pad_end = pad_total - pad_beg
        x = ZeroPadding2D((pad_beg, pad_end))(x)
        return Conv2D(filters,
                      (kernel_size, kernel_size),
                      strides=(stride, stride),
                      padding='valid', use_bias=False,
                      dilation_rate=(rate, rate),
                      name=prefix)(x)

In [4]:
def _xception_block(inputs, depth_list, prefix, skip_connection_type, stride,
                    rate=1, depth_activation=False, return_skip=False):
    """ Basic building block of modified Xception network
        Args:
            inputs: input tensor
            depth_list: number of filters in each SepConv layer. len(depth_list) == 3
            prefix: prefix before name
            skip_connection_type: one of {'conv','sum','none'}
            stride: stride at last depthwise conv
            rate: atrous rate for depthwise convolution
            depth_activation: flag to use activation between depthwise & pointwise convs
            return_skip: flag to return additional tensor after 2 SepConvs for decoder
            """
    residual = inputs
    for i in range(3):
        residual = SepConv_BN(residual,
                              depth_list[i],
                              prefix + '_separable_conv{}'.format(i + 1),
                              stride=stride if i == 2 else 1,
                              rate=rate,
                              depth_activation=depth_activation)
        if i == 1:
            skip = residual
    if skip_connection_type == 'conv':
        shortcut = _conv2d_same(inputs, depth_list[-1], prefix + '_shortcut',
                                kernel_size=1,
                                stride=stride)
        shortcut = BatchNormalization(name=prefix + '_shortcut_BN')(shortcut)
        outputs = layers.add([residual, shortcut])
    elif skip_connection_type == 'sum':
        outputs = layers.add([residual, inputs])
    elif skip_connection_type == 'none':
        outputs = residual
    if return_skip:
        return outputs, skip
    else:
        return outputs


In [5]:
def relu6(x):
    return relu(x, max_value=6)


def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def _inverted_res_block(inputs, expansion, stride, alpha, filters, block_id, skip_connection, rate=1):
    in_channels = inputs.shape[-1]  # inputs._keras_shape[-1]
    pointwise_conv_filters = int(filters * alpha)
    pointwise_filters = _make_divisible(pointwise_conv_filters, 8)
    x = inputs
    prefix = 'expanded_conv_{}_'.format(block_id)
    if block_id:
        # Expand

        x = Conv2D(expansion * in_channels, kernel_size=1, padding='same',
                   use_bias=False, activation=None,
                   name=prefix + 'expand')(x)
        x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                               name=prefix + 'expand_BN')(x)
        x = Activation(relu6, name=prefix + 'expand_relu')(x)
    else:
        prefix = 'expanded_conv_'
    # Depthwise
    x = DepthwiseConv2D(kernel_size=3, strides=stride, activation=None,
                        use_bias=False, padding='same', dilation_rate=(rate, rate),
                        name=prefix + 'depthwise')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                           name=prefix + 'depthwise_BN')(x)

    x = Activation(relu6, name=prefix + 'depthwise_relu')(x)

    # Project
    x = Conv2D(pointwise_filters,
               kernel_size=1, padding='same', use_bias=False, activation=None,
               name=prefix + 'project')(x)
    x = BatchNormalization(epsilon=1e-3, momentum=0.999,
                           name=prefix + 'project_BN')(x)

    if skip_connection:
        return Add(name=prefix + 'add')([inputs, x])

    # if in_channels == pointwise_filters and stride == 1:
    #    return Add(name='res_connect_' + str(block_id))([inputs, x])

    return x

In [6]:
def Deeplabv3(weights=None, input_tensor=None, input_shape=(512, 512, 1), classes=2, backbone='mobilenetv2',
              OS=16, alpha=1.):
    """ Instantiates the Deeplabv3+ architecture

    Optionally loads weights pre-trained
    on PASCAL VOC or Cityscapes. This model is available for TensorFlow only.
    # Arguments
        weights: one of 'pascal_voc' (pre-trained on pascal voc),
            'cityscapes' (pre-trained on cityscape) or None (random initialization)
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: shape of input image. format HxWxC
            PASCAL VOC model was trained on (512,512,3) images. None is allowed as shape/width
        classes: number of desired classes. PASCAL VOC has 21 classes, Cityscapes has 19 classes.
            If number of classes not aligned with the weights used, last layer is initialized randomly
        backbone: backbone to use. one of {'xception','mobilenetv2'}
        activation: optional activation to add to the top of the network.
            One of 'softmax', 'sigmoid' or None
        OS: determines input_shape/feature_extractor_output ratio. One of {8,16}.
            Used only for xception backbone.
        alpha: controls the width of the MobileNetV2 network. This is known as the
            width multiplier in the MobileNetV2 paper.
                - If `alpha` < 1.0, proportionally decreases the number
                    of filters in each layer.
                - If `alpha` > 1.0, proportionally increases the number
                    of filters in each layer.
                - If `alpha` = 1, default number of filters from the paper
                    are used at each layer.
            Used only for mobilenetv2 backbone. Pretrained is only available for alpha=1.

    # Returns
        A Keras model instance.

    # Raises
        RuntimeError: If attempting to run this model with a
            backend that does not support separable convolutions.
        ValueError: in case of invalid argument for `weights` or `backbone`

    """

    if not (weights in {'pascal_voc', 'cityscapes', None}):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `pascal_voc`, or `cityscapes` '
                         '(pre-trained on PASCAL VOC)')

    if not (backbone in {'xception', 'mobilenetv2'}):
        raise ValueError('The `backbone` argument should be either '
                         '`xception`  or `mobilenetv2` ')

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        img_input = input_tensor

    if backbone == 'xception':
        if OS == 8:
            entry_block3_stride = 1
            middle_block_rate = 2  # ! Not mentioned in paper, but required
            exit_block_rates = (2, 4)
            atrous_rates = (12, 24, 36)
        else:
            entry_block3_stride = 2
            middle_block_rate = 1
            exit_block_rates = (1, 2)
            atrous_rates = (6, 12, 18)

        x = Conv2D(32, (3, 3), strides=(2, 2),
                   name='entry_flow_conv1_1', use_bias=False, padding='same')(img_input)
        x = BatchNormalization(name='entry_flow_conv1_1_BN')(x)
        x = Activation('relu')(x)

        x = _conv2d_same(x, 64, 'entry_flow_conv1_2', kernel_size=3, stride=1)
        x = BatchNormalization(name='entry_flow_conv1_2_BN')(x)
        x = Activation('relu')(x)

        x = _xception_block(x, [128, 128, 128], 'entry_flow_block1',
                            skip_connection_type='conv', stride=2,
                            depth_activation=False)
        x, skip1 = _xception_block(x, [256, 256, 256], 'entry_flow_block2',
                                   skip_connection_type='conv', stride=2,
                                   depth_activation=False, return_skip=True)

        x = _xception_block(x, [728, 728, 728], 'entry_flow_block3',
                            skip_connection_type='conv', stride=entry_block3_stride,
                            depth_activation=False)
        for i in range(16):
            x = _xception_block(x, [728, 728, 728], 'middle_flow_unit_{}'.format(i + 1),
                                skip_connection_type='sum', stride=1, rate=middle_block_rate,
                                depth_activation=False)

        x = _xception_block(x, [728, 1024, 1024], 'exit_flow_block1',
                            skip_connection_type='conv', stride=1, rate=exit_block_rates[0],
                            depth_activation=False)
        x = _xception_block(x, [1536, 1536, 2048], 'exit_flow_block2',
                            skip_connection_type='none', stride=1, rate=exit_block_rates[1],
                            depth_activation=True)

    else:
        OS = 1
        first_block_filters = _make_divisible(32 * alpha, 8)
        x = Conv2D(first_block_filters,
                   kernel_size=3,
                   strides=(2, 2), padding='same',
                   use_bias=False, name='Conv')(img_input)
        x = BatchNormalization(
            epsilon=1e-3, momentum=0.999, name='Conv_BN')(x)
        x = Activation(relu6, name='Conv_Relu6')(x)

        x = _inverted_res_block(x, filters=16, alpha=alpha, stride=1,
                                expansion=1, block_id=0, skip_connection=False)

        x = _inverted_res_block(x, filters=24, alpha=alpha, stride=2,
                                expansion=6, block_id=1, skip_connection=False)
        x = _inverted_res_block(x, filters=24, alpha=alpha, stride=1,
                                expansion=6, block_id=2, skip_connection=True)

        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=2,
                                expansion=6, block_id=3, skip_connection=False)
        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                                expansion=6, block_id=4, skip_connection=True)
        x = _inverted_res_block(x, filters=32, alpha=alpha, stride=1,
                                expansion=6, block_id=5, skip_connection=True)

        # stride in block 6 changed from 2 -> 1, so we need to use rate = 2
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1,  # 1!
                                expansion=6, block_id=6, skip_connection=False)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=7, skip_connection=True)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=8, skip_connection=True)
        x = _inverted_res_block(x, filters=64, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=9, skip_connection=True)

        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=10, skip_connection=False)
        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=11, skip_connection=True)
        x = _inverted_res_block(x, filters=96, alpha=alpha, stride=1, rate=2,
                                expansion=6, block_id=12, skip_connection=True)

        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=2,  # 1!
                                expansion=6, block_id=13, skip_connection=False)
        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=14, skip_connection=True)
        x = _inverted_res_block(x, filters=160, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=15, skip_connection=True)

        x = _inverted_res_block(x, filters=320, alpha=alpha, stride=1, rate=4,
                                expansion=6, block_id=16, skip_connection=False)

    # end of feature extractor

    # branching for Atrous Spatial Pyramid Pooling

    # Image Feature branch
    shape_before = tf.shape(x)
    b4 = GlobalAveragePooling2D()(x)
    # from (b_size, channels)->(b_size, 1, 1, channels)
    b4 = Lambda(lambda x: K.expand_dims(x, 1))(b4)
    b4 = Lambda(lambda x: K.expand_dims(x, 1))(b4)
    b4 = Conv2D(256, (1, 1), padding='same',
                use_bias=False, name='image_pooling')(b4)
    b4 = BatchNormalization(name='image_pooling_BN', epsilon=1e-5)(b4)
    b4 = Activation('relu')(b4)
    # upsample. have to use compat because of the option align_corners
    size_before = tf.keras.backend.int_shape(x)
    b4 = Lambda(lambda x: tf.compat.v1.image.resize(x, size_before[1:3],
                                                    method='bilinear', align_corners=True))(b4)
    # simple 1x1
    b0 = Conv2D(256, (1, 1), padding='same', use_bias=False, name='aspp0')(x)
    b0 = BatchNormalization(name='aspp0_BN', epsilon=1e-5)(b0)
    b0 = Activation('relu', name='aspp0_activation')(b0)

    # there are only 2 branches in mobilenetV2. not sure why
    if backbone == 'xception':
        # rate = 6 (12)
        b1 = SepConv_BN(x, 256, 'aspp1',
                        rate=atrous_rates[0], depth_activation=True, epsilon=1e-5)
        # rate = 12 (24)
        b2 = SepConv_BN(x, 256, 'aspp2',
                        rate=atrous_rates[1], depth_activation=True, epsilon=1e-5)
        # rate = 18 (36)
        b3 = SepConv_BN(x, 256, 'aspp3',
                        rate=atrous_rates[2], depth_activation=True, epsilon=1e-5)

        # concatenate ASPP branches & project
        x = Concatenate()([b4, b0, b1, b2, b3])
    else:
        x = Concatenate()([b4, b0])

    x = Conv2D(256, (1, 1), padding='same',
               use_bias=False, name='concat_projection')(x)
    x = BatchNormalization(name='concat_projection_BN', epsilon=1e-5)(x)
    x = Activation('relu')(x)
    x = Dropout(0.1)(x)
    # print(x.shape)
    # DeepLab v.3+ decoder

    if backbone == 'xception':
        # Feature projection
        # x4 (x2) block
        size_before2 = tf.keras.backend.int_shape(x)
        # print(size_before2)
        x = Lambda(lambda xx: tf.compat.v1.image.resize(xx,
                                                        size_before2[1:3] * tf.constant(OS // 4),
                                                        method='bilinear', align_corners=True))(x)
        
        # print(x.shape)

        dec_skip1 = Conv2D(48, (1, 1), padding='same',
                           use_bias=False, name='feature_projection0')(skip1)
        dec_skip1 = BatchNormalization(
            name='feature_projection0_BN', epsilon=1e-5)(dec_skip1)
        dec_skip1 = Activation('relu')(dec_skip1)
        x = Concatenate()([x, dec_skip1])
        x = SepConv_BN(x, 256, 'decoder_conv0',
                       depth_activation=True, epsilon=1e-5)
        x = SepConv_BN(x, 256, 'decoder_conv1',
                       depth_activation=True, epsilon=1e-5)

    # you can use it with arbitary number of classes
    # print(weights)
    # print(classes)
    if (weights == 'pascal_voc' and classes == 21) or (weights == 'cityscapes' and classes == 19):
        last_layer_name = 'logits_semantic'
    else:
        last_layer_name = 'custom_logits_semantic'

    x = Conv2D(classes, (1, 1), padding='same', name=last_layer_name)(x)
    size_before3 = tf.keras.backend.int_shape(img_input)
    x = Lambda(lambda xx: tf.compat.v1.image.resize(xx,
                                                    size_before3[1:3],
                                                    method='bilinear', align_corners=True))(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # if activation in {'softmax', 'sigmoid'}:
    #    x = tf.keras.layers.Activation(activation)(x)
    
    x = tf.keras.layers.Activation('softmax')(x)
    
    print(x.shape)
    print(inputs.shape)
    

    # predictions = tf.keras.layers.Lambda(lambda x: tf.cast(tf.argmax(x, axis=-1), dtype='int64'))(x)
    
    # model = Model(inputs, predictions, name='deeplabv3plus')
    # x = x[..., 1]
    model = Model(inputs, x, name='deeplabv3plus')

    # load weights

    if weights == 'pascal_voc':
        if backbone == 'xception':
            weights_path = get_file('deeplabv3_xception_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH_X,
                                    cache_subdir='models')
        else:
            weights_path = get_file('deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH_MOBILE,
                                    cache_subdir='models')
        model.load_weights(weights_path, by_name=True)
    elif weights == 'cityscapes':
        if backbone == 'xception':
            weights_path = get_file('deeplabv3_xception_tf_dim_ordering_tf_kernels_cityscapes.h5',
                                    WEIGHTS_PATH_X_CS,
                                    cache_subdir='models')
        else:
            weights_path = get_file('deeplabv3_mobilenetv2_tf_dim_ordering_tf_kernels_cityscapes.h5',
                                    WEIGHTS_PATH_MOBILE_CS,
                                    cache_subdir='models')
        model.load_weights(weights_path, by_name=True)
    
    return model


In [7]:
def dice_coefficient(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def iou(y_true, y_pred, smooth=1):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    union = np.sum(y_true_f) + np.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)

def save_image(image, filepath):
    image = (image * 255).astype(np.uint8)  # Scale image to 0-255
    tiff.imwrite(filepath, image)

def predict1(model, mask_dir, image_dir, output_dir, model_type, number=-1):
    dice_scores = []
    iou_scores = []
    precisions = []
    recalls = []
    f1_scores = []

    os.makedirs(output_dir, exist_ok=True)

    mask_files = [f for f in os.listdir(mask_dir) if f.endswith('_resized.tif')]
    i = 0
    print(len(mask_files))
    for mask_file in mask_files:
        # try:
        # Extract the number i from the mask file name
        i_str = mask_file.split('_')[2]
        print(i_str)
        image_file = f"{i_str}.tif"

        mask_path = os.path.join(mask_dir, mask_file)
        image_path=os.path.join(image_dir, image_file)

        # Load the image and mask
        arbitrary_img = tiff.imread(image_path)
        arbitrary_mask = tiff.imread(mask_path)

        # Ensure the image is in the correct shape (512, 512, 1)
        if len(arbitrary_img.shape) == 2:
            arbitrary_img = np.expand_dims(arbitrary_img, axis=-1)
        elif arbitrary_img.shape[0] == 2:
            arbitrary_img = arbitrary_img[0]  # assuming you need the first channel
        
        predicted_mask = model.predict(np.expand_dims(arbitrary_img, axis=0))[0]
        print(predicted_mask.shape)
        # print(predicted_mask)

        # Apply thresholding to predicted mask
        predicted_mask_thresh = (predicted_mask > 0.5).astype(np.uint8)

        # Save predicted mask images
        # save_image(predicted_mask, f"./{output_dir}/{model_type}_Predicted_Image_{i_str}.tif")
        # save_image(predicted_mask_thresh, f"./{output_dir}/{model_type}_{i_str}.tif")

        # Plot and save the arbitrary image, actual mask, and predicted mask
        plt.figure(figsize=(10, 5))

        plt.subplot(2, 2, 1)
        plt.imshow(arbitrary_img.squeeze(), cmap='gray')
        plt.title('Arbitrary Image')
        plt.axis('off')

        plt.subplot(2, 2, 2)
        plt.imshow(arbitrary_mask, cmap='gray')
        plt.title('Actual Mask')
        plt.axis('off')

        plt.subplot(2, 2, 3)
        merged_mask = np.argmax(predicted_mask, axis=-1)
        plt.imshow(merged_mask, cmap='gray')
        plt.title('Predicted Mask without thresholding')
        plt.axis('off')

        plt.subplot(2, 2, 4)
        merged_mask_thresh = np.argmax(predicted_mask_thresh, axis=-1)
        plt.imshow(merged_mask_thresh.squeeze(), cmap='gray')
        plt.title('Predicted Mask')
        plt.axis('off')

        plt.savefig(f"./{output_dir}/{model_type}_Visualization_{i_str}.jpg")
        plt.close()

        save_image(merged_mask, f"./{output_dir}/{model_type}_Predicted_Image_{i_str}.tif")
        save_image(merged_mask_thresh, f"./{output_dir}/{model_type}_{i_str}.tif")

        # Calculate metrics
        dice = dice_coefficient(arbitrary_mask, merged_mask_thresh)
        iou_score = iou(arbitrary_mask, merged_mask_thresh)
        precision = precision_score(arbitrary_mask.flatten(), merged_mask_thresh.flatten())
        recall = recall_score(arbitrary_mask.flatten(), merged_mask_thresh.flatten())
        f1 = f1_score(arbitrary_mask.flatten(), merged_mask_thresh.flatten())

        dice_scores.append(dice)
        iou_scores.append(iou_score)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)
    
        i += 1
        if i == number:
            break
    # Print and return the average metrics
    mean_dice = np.mean(dice_scores)
    mean_iou = np.mean(iou_scores)
    mean_precision = np.mean(precisions)
    mean_recall = np.mean(recalls)
    mean_f1 = np.mean(f1_scores)

    print(f"Mean Dice Coefficient: {mean_dice}")
    print(f"Mean IoU: {mean_iou}")
    print(f"Mean Precision: {mean_precision}")
    print(f"Mean Recall: {mean_recall}")
    print(f"Mean F1 Score: {mean_f1}")

    return mean_dice, mean_iou, mean_precision, mean_recall, mean_f1

In [8]:
# Define paths to your dataset

# Parameters
NUM_CLASSES = 2  # Number of segmentation classes
BACKBONE = 'mobilenetv2'  # Backbone network
BATCH_SIZE = 8
EPOCHS = 20
LEARNING_RATE = 0.001
OUTPUT_STRIDE = 8
# ACTIVATION = 'sigmoid'  # Use 'sigmoid' for binary segmentation
VALIDATION_SPLIT = 0.125


In [9]:
import json

def load_data_new(input_dir, mask_dir):
    
    def numerical_sort(value):
        numbers = re.findall(r'\d+', os.path.basename(value))
        return int(numbers[-1]) if numbers else -1

    # Ensure the directories exist
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist.")
        return None, None
    
    if not os.path.exists(mask_dir):
        print(f"Mask directory {mask_dir} does not exist.")
        return None, None

    # Search for .tif files in the directories
    input_files = glob.glob(os.path.join(input_dir, '*.tif'))
    mask_files = glob.glob(os.path.join(mask_dir, '*.tif'))

    # print("Found input files:", input_files)
    # print("Found mask files:", mask_files)

    if not input_files:
        print(f"No input files found in {input_dir}.")
        return None, None

    if not mask_files:
        print(f"No mask files found in {mask_dir}.")
        return None, None

    images = []
    masks = []

    for mask_file in mask_files:
        # Extract the number i from the mask filename
        match = re.search(r'NDWI_Mask_(\d+)_resized_corrupt.tif', os.path.basename(mask_file))
        print(match)
        if match:
            i = match.group(1)
            input_file = os.path.join(input_dir, f'{i}.tif')
            # print("Input File : ",input_file)
            # Check if the corresponding input file exists
            if os.path.exists(input_file):
                # Read input file
                with rasterio.open(input_file) as src:
                    img = src.read(1)  # Read the first band assuming it's a single-band image
                    images.append(img)

                # Read mask file
                with rasterio.open(mask_file) as src:
                    msk = src.read(1)  # Read the first band assuming it's a single-band image
                    masks.append(msk)
            else:
                print(f"Corresponding input file {input_file} for mask {mask_file} not found.")
        # break
    if not images or not masks:
        print("No matching pairs of images and masks found.")
        return None, None

    masks = tf.one_hot(masks, depth=2)
    
    # print(masks)
    return np.array(images), np.array(masks)

In [10]:
corruption = 0
input_dir = './data_new/'
mask_dir = f'./GEE_Masks/GEE_resized/train_gee/train_{corruption}_gee_with_diff_kernels'
output_dir = f'./GEE_Output/Adversarial/New_20_Epoch_DeepLab_{corruption}_with_diff_kernels'

print(mask_dir)
print(output_dir)

os.makedirs(output_dir,exist_ok=True)
# Load data
images, masks = load_data_new(input_dir, mask_dir)

# Define input shape and number of classes
images = np.expand_dims(images, axis=-1)
input_shape = images.shape[1:]

./GEE_Masks/GEE_resized/train_gee/train_0_gee_with_diff_kernels
./GEE_Output/Adversarial/New_20_Epoch_DeepLab_0_with_diff_kernels
<re.Match object; span=(0, 31), match='NDWI_Mask_0_resized_corrupt.tif'>


c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\rasterio\__init__.py:368: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


<re.Match object; span=(0, 34), match='NDWI_Mask_1000_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1001_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1002_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1003_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1004_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1005_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1006_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1007_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1009_resized_corrupt.tif'>
<re.Match object; span=(0, 33), match='NDWI_Mask_100_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1011_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1013_resized_corrupt.tif'>
<re.Match object; span=(0, 34), match='NDWI_Mask_1014_resized_corrupt.tif'>
<re.Match obj

In [11]:
# Model setup
INPUT_SHAPE = input_shape

model = Deeplabv3(
    input_shape=INPUT_SHAPE,
    classes=NUM_CLASSES,
    backbone=BACKBONE,
    weights="cityscapes"
)

print(images.shape)
print(masks.shape)


(None, 512, 512, 2)
(None, 512, 512, 1)
8984520/8984520 [==============================] - 1s 0us/step


ValueError: Shape mismatch in layer #1 (named Conv) for weight Conv/kernel:0. Weight expects shape (3, 3, 1, 32). Received saved weight with shape (32, 3, 3, 3)

In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="categorical_crossentropy",
    metrics=['accuracy']
)

# Callbacks
checkpoint = ModelCheckpoint("deeplabv3_best_model.h5", save_best_only=True, monitor="val_loss", mode="min")
# early_stopping = EarlyStopping(patience=5, monitor="val_loss", mode="min", restore_best_weights=True)

model.fit(images, masks, validation_split=VALIDATION_SPLIT, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[checkpoint])
model.save(f"deeplabv3_20_epoch_{corruption}_trained_model.h5")

model_name = f'DeepLab_GEE_20_epoch_{corruption}_with_diff_kernels'
print(model_name)
model.save(model_name)

Epoch 1/20
111/111 [==============================] - 67s 489ms/step - loss: 0.2672 - accuracy: 0.9010 - val_loss: 0.5144 - val_accuracy: 0.8172
Epoch 2/20
111/111 [==============================] - 55s 494ms/step - loss: 0.1964 - accuracy: 0.9342 - val_loss: 0.5731 - val_accuracy: 0.8172
Epoch 3/20
111/111 [==============================] - 56s 503ms/step - loss: 0.1853 - accuracy: 0.9380 - val_loss: 0.5960 - val_accuracy: 0.8172
Epoch 4/20
111/111 [==============================] - 56s 507ms/step - loss: 0.1743 - accuracy: 0.9418 - val_loss: 0.6468 - val_accuracy: 0.8172
Epoch 5/20
111/111 [==============================] - 55s 500ms/step - loss: 0.1680 - accuracy: 0.9441 - val_loss: 0.6422 - val_accuracy: 0.8172
Epoch 6/20
111/111 [==============================] - 56s 504ms/step - loss: 0.1679 - accuracy: 0.9452 - val_loss: 0.7417 - val_accuracy: 0.8172
Epoch 7/20
111/111 [==============================] - 56s 502ms/step - loss: 0.1641 - accuracy: 0.9457 - val_loss: 0.5993 - val_ac

INFO:tensorflow:Assets written to: DeepLab_GEE_20_epoch_0_with_diff_kernels\assets


INFO:tensorflow:Assets written to: DeepLab_GEE_20_epoch_0_with_diff_kernels\assets


In [ ]:
mask_dir = './GEE_Masks/GEE_resized/test_gee/'

In [ ]:
# model_name = 'UNet_dense_GEE_20_epoch_22_with_diff_kernels'
model = tf.keras.models.load_model(model_name, compile=False)
model.compile()
predict1(model,mask_dir,input_dir,output_dir,BACKBONE)

253
1008
1/1 [==============================] - 1s 640ms/step
(512, 512, 2)


c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1010
1/1 [==============================] - 0s 20ms/step
(512, 512, 2)


c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


1012
1/1 [==============================] - 0s 20ms/step
(512, 512, 2)


...

986
1/1 [==============================] - 0s 20ms/step
(512, 512, 2)


c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


9
1/1 [==============================] - 0s 20ms/step
(512, 512, 2)
Mean Dice Coefficient: 0.08636627829703292
Mean IoU: 0.08636627829703292
Mean Precision: 0.0
Mean Recall: 0.0
Mean F1 Score: 0.0


c:\Users\ADMIN\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


(0.08636627829703292, 0.08636627829703292, 0.0, 0.0, 0.0)